In [ ]:
import requests
import pandas as pd
import geopandas as gp
import numpy as np
from pathlib import Path

### Helpers

In [ ]:
outdir = Path('../../../../data/intermediate/')

In [ ]:
def safelyget(alist, idx, default='N/A'):
    """Returns alist[idx] if exists, else default"""
    try:
        return alist[idx]
    except (KeyError, IndexError):
        return default

### Get all ACIS stations that have avg daily temp data, with their data validity rage, ID and lat/lon

Variable 13 = snow water equivalent

In [ ]:
baseurl = 'http://data.rcc-acis.org/StnMeta'
params = {
    'state': "AK",
    'meta': "name,uid,ll,elev,valid_daterange",
    'elems': "avgt,snow"
}

In [ ]:
resp = requests.get(url=baseurl, params=params)

In [ ]:
stationdat = resp.json()['meta']
len(stationdat)

1072

In [ ]:
stationdat

[{'ll': [-148.91667, 63.73333],
  'valid_daterange': [['1949-09-01', '1950-02-28'],
   ['1949-09-01', '1950-02-28']],
  'name': 'MCKINLEY NATIONAL PARK AP',
  'elev': 1720.0,
  'uid': 1},
 {'ll': [-152.29694, 60.81417],
  'valid_daterange': [['1978-01-01', '2010-11-30'],
   ['1978-01-01', '2010-11-30']],
  'name': 'BIG RIVER LAKES',
  'elev': 60.0,
  'uid': 450},
 {'ll': [179.25, 51.38333],
  'valid_daterange': [['1949-01-01', '1965-10-29'],
   ['1949-01-01', '1950-11-20']],
  'name': 'AMCHITKA ISLAND',
  'uid': 20416},
 {'ll': [179.28333, 51.38333],
  'valid_daterange': [['1943-02-01', '1993-08-31'],
   ['1943-11-01', '1993-08-31']],
  'name': 'AMCHITKA',
  'elev': 223.0,
  'uid': 20418},
 {'ll': [-178.03333, 51.75],
  'valid_daterange': [['1948-09-04', '1949-03-16'],
   ['1948-09-04', '1949-03-16']],
  'name': 'ASI TANAGA ISLAND NS',
  'uid': 20425},
 {'ll': [-176.64999, 51.88333],
  'valid_daterange': [['1949-01-01', '1950-06-30'],
   ['1949-01-01', '1950-06-30']],
  'name': 'ADAK D

In [ ]:
records = [   
    dict(
        name=stat['name'],
        acisID=stat['uid'],
        longitude=safelyget(safelyget(stat, 'll', []), 0, np.nan),
        latitude=safelyget(safelyget(stat, 'll', []), 1, np.nan),
        elev_ft=safelyget(stat, 'elev', np.nan), 
        valid_avgT_start=safelyget(stat['valid_daterange'][0], 0, ''),
        valid_avgT_end=safelyget(stat['valid_daterange'][0], 1, ''),
        valid_snowdepth_start=safelyget(stat['valid_daterange'][1], 0, ''),
        valid_snowdepth_end=safelyget(stat['valid_daterange'][1], 1, ''),
    )
    for stat in stationdat
]


In [ ]:
stationDF = pd.DataFrame.from_records(records)
stationDF

,name,acisID,longitude,latitude,elev_ft,valid_avgT_start,valid_avgT_end,valid_snowdepth_start,valid_snowdepth_end
0,MCKINLEY NATIONAL PARK AP,1,-148.91667,63.73333,1720.0,1949-09-01,1950-02-28,1949-09-01,1950-02-28
1,BIG RIVER LAKES,450,-152.29694,60.81417,60.0,1978-01-01,2010-11-30,1978-01-01,2010-11-30
2,AMCHITKA ISLAND,20416,179.25000,51.38333,NaN,1949-01-01,1965-10-29,1949-01-01,1950-11-20
3,AMCHITKA,20418,179.28333,51.38333,223.0,1943-02-01,1993-08-31,1943-11-01,1993-08-31
4,ASI TANAGA ISLAND NS,20425,-178.03333,51.75000,NaN,1948-09-04,1949-03-16,1948-09-04,1949-03-16
...,...,...,...,...,...,...,...,...,...
1067,FAIRBANKS 18.4 WNW,100836,-148.21570,64.95090,1379.0,,,2021-11-05,2023-02-18
1068,MOSQUITO LAKE 5.7 SSE,100978,-136.02306,59.45450,214.0,,,2022-01-19,2023-02-18
1069,HAINES 0.3 N,101176,-135.44055,59.24437,344.0,,,2020-04-17,2022-11-10
1070,HAINES 0.5 NNE,101585,-135.43327,59.24731,371.0,,,2021-01-27,2023-02-03


Let's drop stations with no valid temperature data as well as too short / old temp series

In [ ]:
stationDF = stationDF[~(stationDF.valid_avgT_start=='')] 
stationDF = stationDF[stationDF.valid_avgT_end > '1990']
stationDF = stationDF.sort_values(by='name')
stationDF = stationDF.reset_index(drop=True)
stationDF

,name,acisID,longitude,latitude,elev_ft,valid_avgT_start,valid_avgT_end,valid_snowdepth_start,valid_snowdepth_end
0,ADAK AP,20427,-176.64999,51.88334,17.0,1942-10-01,2023-02-20,1942-11-01,1996-03-31
1,AKUTAN,21171,-165.76666,54.13333,NaN,1986-01-01,1990-02-22,1986-01-01,1990-02-22
2,ALASKA PACIFIC UNIVERSITY,21216,-149.80556,61.18889,220.0,1993-07-01,2004-10-31,1993-07-01,2004-10-31
3,ALCAN HWY MI-1244 ALASKA,65647,-141.46671,62.81670,1800.0,1990-06-18,2022-11-28,,
4,ALEKNAGIK 1 NNE,93859,-158.61450,59.28400,80.0,2019-10-11,2023-02-21,,
...,...,...,...,...,...,...,...,...,...
604,YAKUTAT AIRPORT,20602,-139.67104,59.51211,41.0,1917-04-01,2023-02-20,1917-04-01,2019-08-28
605,YAKUTAT POWER,94427,-139.85556,59.72861,71.0,2019-12-01,2020-03-01,2019-12-01,2020-02-12
606,YANTARNI BAY ALASKA,83027,-157.03329,56.91440,96.0,2010-08-26,2019-12-05,,
607,Yakutat Area,32886,NaN,NaN,NaN,1917-05-01,2023-02-20,1917-08-01,2019-08-28


In [ ]:
stationDF.to_csv(outdir / "ACIS_stations_AK_cleaned.csv")

In [ ]:
selectedstationsDF = stationDF[
    (stationDF.valid_avgT_start < '1980-01-01') & (stationDF.valid_avgT_end > '2023-02-01')]
selectedstationsDF

,name,acisID,longitude,latitude,elev_ft,valid_avgT_start,valid_avgT_end,valid_snowdepth_start,valid_snowdepth_end
0,ADAK AP,20427,-176.64999,51.88334,17.0,1942-10-01,2023-02-20,1942-11-01,1996-03-31
8,ALYESKA,20684,-149.11079,60.95830,272.0,1963-11-07,2023-02-21,1963-11-07,2023-02-21
14,ANAKTUVUK AUTO,21055,-151.76666,68.16667,2100.0,1953-07-11,2023-02-20,1953-07-09,1973-05-31
26,ANCHORAGE MERRILL FIELD,20717,-149.85724,61.21630,126.0,1914-11-01,2023-02-20,1916-02-16,2005-02-20
29,ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT,20705,-150.02771,61.16916,125.0,1952-04-01,2023-02-20,1953-02-01,2023-02-20
...,...,...,...,...,...,...,...,...,...
591,WHITE MOUNTAIN,20942,-163.39999,64.68333,20.0,1922-01-05,2023-02-20,1922-01-14,1990-05-31
599,WISEMAN,21043,-150.10957,67.40822,1181.0,1919-11-01,2023-02-21,1918-11-01,2023-02-20
602,WRANGELL AP,29707,-132.38741,56.47320,56.0,1917-11-02,2023-02-10,1917-12-05,2013-02-19
604,YAKUTAT AIRPORT,20602,-139.67104,59.51211,41.0,1917-04-01,2023-02-20,1917-04-01,2019-08-28


In [ ]:
selectedstationsDF.to_csv(outdir / "ACIS_stations_AK_recentTemps.csv")

In [ ]:
stationsGDF = gp.GeoDataFrame(
    stationDF, geometry=gp.points_from_xy(stationDF.longitude, stationDF.latitude))
stationsGDF = stationsGDF.set_crs('epsg:4326')
stationsGDF

/Users/chris/miniconda3/envs/python38_geo/lib/python3.8/site-packages/geopandas/array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)


,name,acisID,longitude,latitude,elev_ft,valid_avgT_start,valid_avgT_end,valid_snowdepth_start,valid_snowdepth_end,geometry
0,ADAK AP,20427,-176.64999,51.88334,17.0,1942-10-01,2023-02-20,1942-11-01,1996-03-31,POINT (-176.64999 51.88334)
1,AKUTAN,21171,-165.76666,54.13333,NaN,1986-01-01,1990-02-22,1986-01-01,1990-02-22,POINT (-165.76666 54.13333)
2,ALASKA PACIFIC UNIVERSITY,21216,-149.80556,61.18889,220.0,1993-07-01,2004-10-31,1993-07-01,2004-10-31,POINT (-149.80556 61.18889)
3,ALCAN HWY MI-1244 ALASKA,65647,-141.46671,62.81670,1800.0,1990-06-18,2022-11-28,,,POINT (-141.46671 62.81670)
4,ALEKNAGIK 1 NNE,93859,-158.61450,59.28400,80.0,2019-10-11,2023-02-21,,,POINT (-158.61450 59.28400)
...,...,...,...,...,...,...,...,...,...,...
604,YAKUTAT AIRPORT,20602,-139.67104,59.51211,41.0,1917-04-01,2023-02-20,1917-04-01,2019-08-28,POINT (-139.67104 59.51211)
605,YAKUTAT POWER,94427,-139.85556,59.72861,71.0,2019-12-01,2020-03-01,2019-12-01,2020-02-12,POINT (-139.85556 59.72861)
606,YANTARNI BAY ALASKA,83027,-157.03329,56.91440,96.0,2010-08-26,2019-12-05,,,POINT (-157.03329 56.91440)
607,Yakutat Area,32886,NaN,NaN,NaN,1917-05-01,2023-02-20,1917-08-01,2019-08-28,POINT EMPTY


In [ ]:
stationsGDF

,name,acisID,longitude,latitude,elev_ft,valid_avgT_start,valid_avgT_end,valid_snowdepth_start,valid_snowdepth_end,geometry
0,ADAK AP,20427,-176.64999,51.88334,17.0,1942-10-01,2023-02-20,1942-11-01,1996-03-31,POINT (-176.64999 51.88334)
1,AKUTAN,21171,-165.76666,54.13333,NaN,1986-01-01,1990-02-22,1986-01-01,1990-02-22,POINT (-165.76666 54.13333)
2,ALASKA PACIFIC UNIVERSITY,21216,-149.80556,61.18889,220.0,1993-07-01,2004-10-31,1993-07-01,2004-10-31,POINT (-149.80556 61.18889)
3,ALCAN HWY MI-1244 ALASKA,65647,-141.46671,62.81670,1800.0,1990-06-18,2022-11-28,,,POINT (-141.46671 62.81670)
4,ALEKNAGIK 1 NNE,93859,-158.61450,59.28400,80.0,2019-10-11,2023-02-21,,,POINT (-158.61450 59.28400)
...,...,...,...,...,...,...,...,...,...,...
604,YAKUTAT AIRPORT,20602,-139.67104,59.51211,41.0,1917-04-01,2023-02-20,1917-04-01,2019-08-28,POINT (-139.67104 59.51211)
605,YAKUTAT POWER,94427,-139.85556,59.72861,71.0,2019-12-01,2020-03-01,2019-12-01,2020-02-12,POINT (-139.85556 59.72861)
606,YANTARNI BAY ALASKA,83027,-157.03329,56.91440,96.0,2010-08-26,2019-12-05,,,POINT (-157.03329 56.91440)
607,Yakutat Area,32886,NaN,NaN,NaN,1917-05-01,2023-02-20,1917-08-01,2019-08-28,POINT EMPTY


In [ ]:
stationsGDF.to_file(outdir / "ACISAK_stations.gpkg", driver='GPKG')

/Users/chris/miniconda3/envs/python38_geo/lib/python3.8/site-packages/geopandas/io/file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
